In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet

In [3]:
df = pd.read_csv('/content/DL_Complete_Yearly_Demand_Profile_train.csv')
# df = pd.read_csv('/content/DL_Yearly_Demand_Profile_train.csv')
df = df.drop('Unnamed: 0',axis =1)
df.head()

,DateTime,State,Hourly Demand Met (in MW),Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2022-01-01,Delhi - 2022,2148.22,8,7 °C,93,CALM,0 km/h,0 km/h,992.62 hPa,0.0 mm,Fog
1,2022-01-01 01:00:00,Delhi - 2022,1863.54,8,7 °C,93,CALM,0 km/h,0 km/h,993.59 hPa,0.0 mm,Fog
2,2022-01-01 02:00:00,Delhi - 2022,1697.53,7,6 °C,93,CALM,0 km/h,0 km/h,993.59 hPa,0.0 mm,Fog
3,2022-01-01 03:00:00,Delhi - 2022,1593.03,7,6 °C,93,W,6 km/h,0 km/h,994.56 hPa,0.0 mm,Fog
4,2022-01-01 04:00:00,Delhi - 2022,1603.47,9,8 °C,93,W,6 km/h,0 km/h,994.56 hPa,0.0 mm,Fog


In [4]:
df = df.rename(columns={'DateTime': 'ds', 'Hourly Demand Met (in MW)': 'y'})

df['y'] = pd.to_numeric(df['y'])
df['ds'] = pd.to_datetime(df['ds'])
df.head()

,ds,State,y,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2022-01-01 00:00:00,Delhi - 2022,2148.22,8,7 °C,93,CALM,0 km/h,0 km/h,992.62 hPa,0.0 mm,Fog
1,2022-01-01 01:00:00,Delhi - 2022,1863.54,8,7 °C,93,CALM,0 km/h,0 km/h,993.59 hPa,0.0 mm,Fog
2,2022-01-01 02:00:00,Delhi - 2022,1697.53,7,6 °C,93,CALM,0 km/h,0 km/h,993.59 hPa,0.0 mm,Fog
3,2022-01-01 03:00:00,Delhi - 2022,1593.03,7,6 °C,93,W,6 km/h,0 km/h,994.56 hPa,0.0 mm,Fog
4,2022-01-01 04:00:00,Delhi - 2022,1603.47,9,8 °C,93,W,6 km/h,0 km/h,994.56 hPa,0.0 mm,Fog


In [5]:
df.ds.dtype

dtype('<M8[ns]')

In [6]:
m = Prophet()
m.add_regressor('Temperature')
m.add_regressor('Humidity')
m.fit(df)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyophf4vu/un6u3tpc.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyophf4vu/dzmpjl6q.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=63573', 'data', 'file=/tmp/tmpyophf4vu/un6u3tpc.json', 'init=/tmp/tmpyophf4vu/dzmpjl6q.json', 'output', 'file=/tmp/tmpyophf4vu/prophet_model5gzz9ybr/prophet_model-20240118051147.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
05:11:47 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
05:11:47 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [29]:
temp = df['Temperature']
humidity = df['Humidity']
#Feed through weather API
future_temp = np.random.uniform(9, 11, 72)
future_temp = pd.Series(future_temp)

temp_series = pd.concat([temp, future_temp], ignore_index=True)

future_humidity = np.random.uniform(50, 100, 72)
future_humidity = pd.Series(future_humidity)

humidity_series = pd.concat([humidity, future_humidity], ignore_index=True)
print(temp)

0       8
1       8
2       7
3       7
4       9
       ..
714    15
715    14
716    13
717    13
718    12
Name: Temperature, Length: 719, dtype: int64


In [30]:
future = m.make_future_dataframe(periods=72,freq='H')

future['Temperature'] = temp_series
future['Humidity'] = humidity_series

In [31]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()



,ds,yhat,yhat_lower,yhat_upper
756,2022-02-03 13:00:00,4044.973266,3805.979781,4267.219487
757,2022-02-03 14:00:00,3736.597030,3494.564450,3960.638016
758,2022-02-03 15:00:00,3522.418812,3288.468409,3765.677059
759,2022-02-03 16:00:00,3450.086263,3219.594888,3678.547399
760,2022-02-03 17:00:00,3505.226797,3261.883112,3736.522728


In [32]:
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Humidity,Humidity_lower,Humidity_upper,Temperature,...,extra_regressors_additive,extra_regressors_additive_lower,extra_regressors_additive_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-01-01 00:00:00,3126.555826,2025.086773,2478.388795,3126.555826,3126.555826,0.268672,0.268672,0.268672,-19.430040,...,-19.161369,-19.161369,-19.161369,31.211034,31.211034,31.211034,0.0,0.0,0.0,2240.147600
1,2022-01-01 01:00:00,3124.381388,1698.557299,2153.625955,3124.381388,3124.381388,0.268672,0.268672,0.268672,-19.430040,...,-19.161369,-19.161369,-19.161369,9.484512,9.484512,9.484512,0.0,0.0,0.0,1931.136681
2,2022-01-01 02:00:00,3122.206950,1464.105533,1944.046175,3122.206950,3122.206950,0.268672,0.268672,0.268672,-23.528075,...,-23.259403,-23.259403,-23.259403,-12.737027,-12.737027,-12.737027,0.0,0.0,0.0,1703.299546
3,2022-01-01 03:00:00,3120.032512,1322.677154,1788.436350,3120.032512,3120.032512,0.268672,0.268672,0.268672,-23.528075,...,-23.259403,-23.259403,-23.259403,-35.329548,-35.329548,-35.329548,0.0,0.0,0.0,1556.162999
4,2022-01-01 04:00:00,3117.858074,1304.607780,1780.171151,3117.858074,3117.858074,0.268672,0.268672,0.268672,-15.332006,...,-15.063334,-15.063334,-15.063334,-58.165881,-58.165881,-58.165881,0.0,0.0,0.0,1529.274276


In [33]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [34]:
plot_components_plotly(m, forecast,)



In [ ]:
from prophet.serialize import model_to_json, model_from_json

with open('serialized_model.json', 'w') as fout:
    fout.write(model_to_json(m))  # Save model

# with open('serialized_model.json', 'r') as fin:
#     m = model_from_json(fin.read())  # Load model


In [35]:
!pip install joblib


In [36]:
import joblib

joblib.dump(m, 'prophet_model_mlr.h5')
# loaded_model = joblib.load('prophet_model.h5')

['prophet_model_mlr.h5']

In [6]:
df = pd.read_csv('/content/KL_Complete_Yearly_Demand_Profile_train.csv')
df = df.drop('Unnamed: 0',axis =1)
df.head()

,DateTime,State,Hourly Demand Met (in MW),Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2022-01-01 01:00:00,Kerala - 2022,2769.91,24,21 °C,83,CALM,0 km/h,0 km/h,"1,001.04 hPa",0.0 mm,Fair
1,2022-01-01 02:00:00,Kerala - 2022,2695.07,23,21 °C,88,ENE,9 km/h,0 km/h,"1,002.03 hPa",0.0 mm,Fair
2,2022-01-01 03:00:00,Kerala - 2022,2615.97,25,21 °C,78,E,9 km/h,0 km/h,"1,002.03 hPa",0.0 mm,Fair
3,2022-01-01 04:00:00,Kerala - 2022,2591.99,27,21 °C,70,E,9 km/h,0 km/h,"1,003.02 hPa",0.0 mm,Fair
4,2022-01-01 05:00:00,Kerala - 2022,2596.73,29,22 °C,66,E,6 km/h,0 km/h,993.14 hPa,0.0 mm,Fair


In [7]:
df = df.rename(columns={'DateTime': 'ds', 'Hourly Demand Met (in MW)': 'y'})

df['y'] = pd.to_numeric(df['y'])
df['ds'] = pd.to_datetime(df['ds'])
df.head()

,ds,State,y,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2022-01-01 01:00:00,Kerala - 2022,2769.91,24,21 °C,83,CALM,0 km/h,0 km/h,"1,001.04 hPa",0.0 mm,Fair
1,2022-01-01 02:00:00,Kerala - 2022,2695.07,23,21 °C,88,ENE,9 km/h,0 km/h,"1,002.03 hPa",0.0 mm,Fair
2,2022-01-01 03:00:00,Kerala - 2022,2615.97,25,21 °C,78,E,9 km/h,0 km/h,"1,002.03 hPa",0.0 mm,Fair
3,2022-01-01 04:00:00,Kerala - 2022,2591.99,27,21 °C,70,E,9 km/h,0 km/h,"1,003.02 hPa",0.0 mm,Fair
4,2022-01-01 05:00:00,Kerala - 2022,2596.73,29,22 °C,66,E,6 km/h,0 km/h,993.14 hPa,0.0 mm,Fair


In [8]:
m = Prophet()
m.add_regressor('Humidity')
m.fit(df)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp3vo0crc2/re4_gx1h.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp3vo0crc2/tkl9ar7r.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=65737', 'data', 'file=/tmp/tmp3vo0crc2/re4_gx1h.json', 'init=/tmp/tmp3vo0crc2/tkl9ar7r.json', 'output', 'file=/tmp/tmp3vo0crc2/prophet_modelgg14tzp_/prophet_model-20240118105650.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:56:50 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:56:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [9]:
temp = df['Temperature']
humidity = df['Humidity']
#Feed through weather API
future_temp = np.random.uniform(24, 33, 72)
future_temp = pd.Series(future_temp)

temp_series = pd.concat([temp, future_temp], ignore_index=True)

future_humidity = np.random.uniform(50, 90, 72)
future_humidity = pd.Series(future_humidity)

humidity_series = pd.concat([humidity, future_humidity], ignore_index=True)
print(temp)

0      24
1      23
2      25
3      27
4      29
       ..
702    29
703    28
704    27
705    27
706    27
Name: Temperature, Length: 707, dtype: int64


In [10]:
future = m.make_future_dataframe(periods=72,freq='H')

future['Temperature'] = temp_series
future['Humidity'] = humidity_series

In [11]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


,ds,yhat,yhat_lower,yhat_upper
745,2022-02-03 13:00:00,3314.706025,3160.910116,3477.208320
746,2022-02-03 14:00:00,3431.139246,3287.263496,3580.274231
747,2022-02-03 15:00:00,3353.621866,3206.954760,3508.111384
748,2022-02-03 16:00:00,3417.127343,3268.811216,3564.137595
749,2022-02-03 17:00:00,3440.207176,3273.447543,3592.244605


In [12]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [13]:
plot_components_plotly(m, forecast,)

In [14]:
import joblib

joblib.dump(m, 'prophet_model_mlr_KL.h5')
# loaded_model = joblib.load('prophet_model.h5')

['prophet_model_mlr_KL.h5']